<a href="https://colab.research.google.com/github/Cellstrat/Image-Segmentation/blob/master/FCN_aws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# %load main.py
#--------------------------
# USER-SPECIFIED DATA
#CellStrat
#Reference blog : https://medium.com/nanonets/how-to-do-image-segmentation-using-deep-learning-c673cc5862ef
# Github code : https://gist.github.com/khanhnamle1994/e2ff59ddca93c0205ac4e566d40b5e88
# This code is based on Udacity CarND tutorial available at : https://github.com/udacity/CarND-Semantic-Segmentation
# Dataset download : Download the Kitti Road dataset from http://www.cvlibs.net/download.php?file=data_road.zip
# Just with 5 training images and 40 training epochs this program took 6-8 hours to execute.
# Better to run it on the Cloud
#--------------------------

# Tune these parameters

In [0]:
# from google.colab import drive
# drive.mount("/content/gdrive")

# import os
# os.chdir('/content/gdrive/My Drive/Vivek-PC-2015/vsinghal/business/Disrupt4.0/AI Lab/demos/Image Segmentation/FCN')
# !pwd

In [3]:
!pip install Pillow

In [4]:
!pip install scikit-image
# !pip install skimage
!pip install scipy==1.0.0

     |████████████████████████████████| 50.0MB 989kB/s 
ERROR: cvxpy 1.0.15 has requirement scipy>=1.1.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.0
    Uninstalling scipy-1.3.0:
      Successfully uninstalled scipy-1.3.0


In [0]:
# !python -m pip install --user numpy scipy matplotlib ipython jupyter pandas sympy nose

In [0]:
import tensorflow as tf
import helper

tf.reset_default_graph()

In [0]:
'''
You should not edit helper.py as part of your submission.

This file is used primarily to download vgg if it has not yet been,
give you the progress of the download, get batches for your training,
as well as around generating and saving the image outputs.
'''

import re
import random
import numpy as np
import os.path
import scipy.misc
import shutil
import zipfile
import time
import tensorflow as tf
from glob import glob
from urllib.request import urlretrieve
from tqdm import tqdm


class DLProgress(tqdm):
	"""
	Report download progress to the terminal.
	:param tqdm: Information fed to the tqdm library to estimate progress.
	"""
	last_block = 0

	def hook(self, block_num=1, block_size=1, total_size=None):
		"""
		Store necessary information for tracking progress.
		:param block_num: current block of the download
		:param block_size: size of current block
		:param total_size: total download size, if known
		"""
		self.total = total_size
		self.update((block_num - self.last_block) * block_size)  # Updates progress
		self.last_block = block_num


def maybe_download_pretrained_vgg(data_dir):
	"""
	Download and extract pretrained vgg model if it doesn't exist
	:param data_dir: Directory to download the model to
	"""
	vgg_filename = 'vgg.zip'
	vgg_path = os.path.join(data_dir, 'vgg')
	vgg_files = [
		os.path.join(vgg_path, 'variables/variables.data-00000-of-00001'),
		os.path.join(vgg_path, 'variables/variables.index'),
		os.path.join(vgg_path, 'saved_model.pb')]

	missing_vgg_files = [vgg_file for vgg_file in vgg_files if not os.path.exists(vgg_file)]
	if missing_vgg_files:
		# Clean vgg dir
		if os.path.exists(vgg_path):
			shutil.rmtree(vgg_path)
		os.makedirs(vgg_path)

		# Download vgg
		print('Downloading pre-trained vgg model...')
		with DLProgress(unit='B', unit_scale=True, miniters=1) as pbar:
			urlretrieve(
				'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip',
				os.path.join(vgg_path, vgg_filename),
				pbar.hook)

		# Extract vgg
		print('Extracting model...')
		zip_ref = zipfile.ZipFile(os.path.join(vgg_path, vgg_filename), 'r')
		zip_ref.extractall(data_dir)
		zip_ref.close()

		# Remove zip file to save space
		os.remove(os.path.join(vgg_path, vgg_filename))


def gen_batch_function(data_folder, image_shape):
	"""
	Generate function to create batches of training data
	:param data_folder: Path to folder that contains all the datasets
	:param image_shape: Tuple - Shape of image
	:return:
	"""
	def get_batches_fn(batch_size):
		"""
		Create batches of training data
		:param batch_size: Batch Size
		:return: Batches of training data
		"""
		# Grab image and label paths
		image_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))
		label_paths = {
			re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path
			for path in glob(os.path.join(data_folder, 'gt_image_2', '*_road_*.png'))}
		background_color = np.array([255, 0, 0])

		# Shuffle training data
		random.shuffle(image_paths)
		# Loop through batches and grab images, yielding each batch
		for batch_i in range(0, len(image_paths), batch_size):
			images = []
			gt_images = []
			for image_file in image_paths[batch_i:batch_i+batch_size]:
				gt_image_file = label_paths[os.path.basename(image_file)]
				# Re-size to image_shape
				image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
				gt_image = scipy.misc.imresize(scipy.misc.imread(gt_image_file), image_shape)

				# Create "one-hot-like" labels by class
				gt_bg = np.all(gt_image == background_color, axis=2)
				gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
				gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)

				images.append(image)
				gt_images.append(gt_image)

			yield np.array(images), np.array(gt_images)
	return get_batches_fn


def gen_test_output(sess, logits, keep_prob, image_pl, data_folder, image_shape):
	"""
	Generate test output using the test images
	:param sess: TF session
	:param logits: TF Tensor for the logits
	:param keep_prob: TF Placeholder for the dropout keep probability
	:param image_pl: TF Placeholder for the image placeholder
	:param data_folder: Path to the folder that contains the datasets
	:param image_shape: Tuple - Shape of image
	:return: Output for for each test image
	"""
	for image_file in glob(os.path.join(data_folder, 'image_2', '*.png')):
		image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)

		# Run inference
		im_softmax = sess.run(
			[tf.nn.softmax(logits)],
			{keep_prob: 1.0, image_pl: [image]})
		# Splice out second column (road), reshape output back to image_shape
		im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
		# If road softmax > 0.5, prediction is road
		segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
		# Create mask based on segmentation to apply to original image
		mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
		mask = scipy.misc.toimage(mask, mode="RGBA")
		street_im = scipy.misc.toimage(image)
		street_im.paste(mask, box=None, mask=mask)

		yield os.path.basename(image_file), np.array(street_im)


def save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image):
	"""
	Save test images with semantic masks of lane predictions to runs_dir.
	:param runs_dir: Directory to save output images
	:param data_dir: Path to the directory that contains the datasets
	:param sess: TF session
	:param image_shape: Tuple - Shape of image
	:param logits: TF Tensor for the logits
	:param keep_prob: TF Placeholder for the dropout keep probability
	:param input_image: TF Placeholder for the image placeholder
	"""
	# Make folder for current run
	output_dir = os.path.join(runs_dir, str(time.time()))
	if os.path.exists(output_dir):
		shutil.rmtree(output_dir)
	os.makedirs(output_dir)

	# Run NN on test images and save them to HD
	print('Training Finished. Saving test images to: {}'.format(output_dir))
	image_outputs = gen_test_output(
		sess, logits, keep_prob, input_image, os.path.join(data_dir, 'data_road/testing'), image_shape)
	for name, image in image_outputs:
		scipy.misc.imsave(os.path.join(output_dir, name), image)


In [8]:
!wget https://cellstrat.s3.us-east-2.amazonaws.com/FCN.zip

--2019-08-08 10:46:59--  https://cellstrat.s3.us-east-2.amazonaws.com/FCN.zip
Resolving cellstrat.s3.us-east-2.amazonaws.com (cellstrat.s3.us-east-2.amazonaws.com)... 52.219.96.8
Connecting to cellstrat.s3.us-east-2.amazonaws.com (cellstrat.s3.us-east-2.amazonaws.com)|52.219.96.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1487533259 (1.4G) [application/zip]
Saving to: ‘FCN.zip’

FCN.zip             100%[===================>]   1.38G  98.6MB/s    in 15s     

2019-08-08 10:47:14 (95.5 MB/s) - ‘FCN.zip’ saved [1487533259/1487533259]



In [9]:
!ls
!unzip FCN.zip

FCN.zip  sample_data
Archive:  FCN.zip
   creating: FCN/
  inflating: FCN/.gitignore          
   creating: FCN/.ipynb_checkpoints/
  inflating: FCN/.ipynb_checkpoints/desktop.ini  
  inflating: FCN/.ipynb_checkpoints/main-checkpoint.ipynb  
   creating: FCN/data/
  inflating: FCN/data/.gitignore     
   creating: FCN/data/data_road/
  inflating: FCN/data/data_road/desktop.ini  
   creating: FCN/data/data_road/testing/
   creating: FCN/data/data_road/testing/calib/
  inflating: FCN/data/data_road/testing/calib/desktop.ini  
  inflating: FCN/data/data_road/testing/calib/um_000000.txt  
  inflating: FCN/data/data_road/testing/calib/um_000001.txt  
  inflating: FCN/data/data_road/testing/calib/um_000002.txt  
  inflating: FCN/data/data_road/testing/calib/um_000003.txt  
  inflating: FCN/data/data_road/testing/calib/um_000004.txt  
  inflating: FCN/data/data_road/testing/desktop.ini  
   creating: FCN/data/data_road/testing/image_2/
  inflating: FCN/data/data_road/testing/image_2/desktop.i

In [10]:
!ls

FCN  FCN.zip  sample_data


In [0]:
!rm -r FCN/runs
!mkdir FCN/runs

In [13]:
tf.reset_default_graph()

#CellStrat
num_classes = 2
NUMBER_OF_CLASSES = 2
image_shape = (160, 576)
IMAGE_SHAPE = (160, 576)
EPOCHS = 40
BATCH_SIZE = 128
DROPOUT = 0.75

# Specify these directory paths

data_dir = 'FCN/data'
runs_dir = 'FCN/runs'
training_dir ='FCN/data/data_road/training'
vgg_path = 'FCN/data/vgg'

#--------------------------
# PLACEHOLDER TENSORS
#--------------------------

correct_label = tf.placeholder(tf.float32, [None, IMAGE_SHAPE[0], IMAGE_SHAPE[1], NUMBER_OF_CLASSES])
learning_rate = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32)

#--------------------------
# FUNCTIONS
#--------------------------

def load_vgg(sess, vgg_path):
  
  # load the model and weights
  model = tf.saved_model.loader.load(sess, ['vgg16'], vgg_path)

  # Get Tensors to be returned from graph
  graph = tf.get_default_graph()
  image_input = graph.get_tensor_by_name('image_input:0')
  keep_prob = graph.get_tensor_by_name('keep_prob:0')
  layer3 = graph.get_tensor_by_name('layer3_out:0')
  layer4 = graph.get_tensor_by_name('layer4_out:0')
  layer7 = graph.get_tensor_by_name('layer7_out:0')

  return image_input, keep_prob, layer3, layer4, layer7

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
   
    # Use a shorter variable name for simplicity
    layer3, layer4, layer7 = vgg_layer3_out, vgg_layer4_out, vgg_layer7_out

    # Apply 1x1 convolution in place of fully connected layer
    fcn8 = tf.layers.conv2d(layer7, filters=num_classes, kernel_size=1, name="fcn8")

    # Upsample fcn8 with size depth=(4096?) to match size of layer 4 so that we can add skip connection with 4th layer
    fcn9 = tf.layers.conv2d_transpose(fcn8, filters=layer4.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn9")

    # Add a skip connection between current final layer fcn8 and 4th layer
    fcn9_skip_connected = tf.add(fcn9, layer4, name="fcn9_plus_vgg_layer4")

    # Upsample again
    fcn10 = tf.layers.conv2d_transpose(fcn9_skip_connected, filters=layer3.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn10_conv2d")

    # Add skip connection
    fcn10_skip_connected = tf.add(fcn10, layer3, name="fcn10_plus_vgg_layer3")

    # Upsample again
    fcn11 = tf.layers.conv2d_transpose(fcn10_skip_connected, filters=num_classes,
    kernel_size=16, strides=(8, 8), padding='SAME', name="fcn11")

    return fcn11

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
  
  # Reshape 4D tensors to 2D, each row represents a pixel, each column a class
  logits = tf.reshape(nn_last_layer, (-1, num_classes), name="fcn_logits")
  correct_label_reshaped = tf.reshape(correct_label, (-1, num_classes))

  # Calculate distance from actual labels using cross entropy
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label_reshaped[:])
  # Take mean for total loss
  loss_op = tf.reduce_mean(cross_entropy, name="fcn_loss")

  # The model implements this operation to find the weights/parameters that would yield correct pixel labels
  train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op, name="fcn_train_op")

  return logits, train_op, loss_op

def train_nn(sess, epochs, batch_size, get_batches_fn, train_op,
             cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):

  keep_prob_value = 0.5
  learning_rate_value = 0.001
  for epoch in range(epochs):
      # Create function to get batches
      total_loss = 0
      for X_batch, gt_batch in get_batches_fn(batch_size):

          loss, _ = sess.run([cross_entropy_loss, train_op],
          feed_dict={input_image: X_batch, correct_label: gt_batch,
          keep_prob: keep_prob_value, learning_rate:learning_rate_value})

          total_loss += loss;

      print("EPOCH {} ...".format(epoch + 1))
      print("Loss = {:.3f}".format(total_loss))
      print()

def run():
  
  # Download pretrained vgg model
  maybe_download_pretrained_vgg(data_dir)

  # A function to get batches
  get_batches_fn = gen_batch_function(training_dir, image_shape)
  
  with tf.Session() as session:
        
    # Returns the three layers, keep probability and input layer from the vgg architecture
    image_input, keep_prob, layer3, layer4, layer7 = load_vgg(session, vgg_path)

    # The resulting network architecture from adding a decoder on top of the given vgg model
    model_output = layers(layer3, layer4, layer7, num_classes)

    # Returns the output logits, training operation and cost operation to be used
    # - logits: each row represents a pixel, each column a class
    # - train_op: function used to get the right parameters to the model to correctly label the pixels
    # - cross_entropy_loss: function outputting the cost which we are minimizing, lower cost should yield higher accuracy
    logits, train_op, cross_entropy_loss = optimize(model_output, correct_label, learning_rate, num_classes)
    
    # Initialize all variables
    session.run(tf.global_variables_initializer())
    session.run(tf.local_variables_initializer())

    print("Model build successful, starting training")

    # Train the neural network
    train_nn(session, EPOCHS, BATCH_SIZE, get_batches_fn, 
             train_op, cross_entropy_loss, image_input,
             correct_label, keep_prob, learning_rate)

    # Run the model with the test images and save each painted output image (roads painted green)
    save_inference_samples(runs_dir, data_dir, session, image_shape, logits, keep_prob, image_input)
    
    print("All done!")

#--------------------------
# MAIN
#--------------------------
if __name__ == '__main__':
    run()

Model build successful, starting training


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:108: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:108: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:109: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:109: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


EPOCH 1 ...
Loss = 56.018

EPOCH 2 ...
Loss = 22.278

EPOCH 3 ...
Loss = 163.621

EPOCH 4 ...
Loss = 6.275

EPOCH 5 ...
Loss = 4.707

EPOCH 6 ...
Loss = 45.383

EPOCH 7 ...
Loss = 2.727

EPOCH 8 ...
Loss = 2.634

EPOCH 9 ...
Loss = 2.054

EPOCH 10 ...
Loss = 1.625

EPOCH 11 ...
Loss = 1.544

EPOCH 12 ...
Loss = 1.055

EPOCH 13 ...
Loss = 0.957

EPOCH 14 ...
Loss = 0.711

EPOCH 15 ...
Loss = 0.740

EPOCH 16 ...
Loss = 0.523

EPOCH 17 ...
Loss = 0.534

EPOCH 18 ...
Loss = 0.439

EPOCH 19 ...
Loss = 0.419

EPOCH 20 ...
Loss = 0.424

EPOCH 21 ...
Loss = 0.375

EPOCH 22 ...
Loss = 0.326

EPOCH 23 ...
Loss = 0.319

EPOCH 24 ...
Loss = 0.273

EPOCH 25 ...
Loss = 0.264

EPOCH 26 ...
Loss = 0.280

EPOCH 27 ...
Loss = 0.248

EPOCH 28 ...
Loss = 0.208

EPOCH 29 ...
Loss = 0.218

EPOCH 30 ...
Loss = 0.191

EPOCH 31 ...
Loss = 0.194

EPOCH 32 ...
Loss = 0.188

EPOCH 33 ...
Loss = 0.190

EPOCH 34 ...
Loss = 0.155

EPOCH 35 ...
Loss = 0.167

EPOCH 36 ...
Loss = 0.146

EPOCH 37 ...
Loss = 0.151

EPOCH

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:148: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:176: DeprecationWarning: `imsave` is de

All done!
